# ML workflow for  Chronic Kidney Disease

In [1]:
# Imports 
from pipeline_perso import *
from sklearn.pipeline import Pipeline

In [2]:
dtypes = {'pcv':'float64','wc':'float64','rc':'float64'}
data = pd.read_csv('kidney_disease.csv',dtype=dtypes,na_values='\t?')
data = data.dropna()

In [3]:
data

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
9,9,53.0,90.0,1.020,2.0,0.0,abnormal,abnormal,present,notpresent,...,29.0,12100.0,3.7,yes,yes,no,poor,no,yes,ckd
11,11,63.0,70.0,1.010,3.0,0.0,abnormal,abnormal,present,notpresent,...,32.0,4500.0,3.8,yes,yes,no,poor,yes,no,ckd
14,14,68.0,80.0,1.010,3.0,2.0,normal,abnormal,present,present,...,16.0,11000.0,2.6,yes,yes,yes,poor,yes,no,ckd
20,20,61.0,80.0,1.015,2.0,0.0,abnormal,abnormal,notpresent,notpresent,...,24.0,9200.0,3.2,yes,yes,yes,poor,yes,yes,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,47.0,6700.0,4.9,no,no,no,good,no,no,notckd
396,396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,54.0,7800.0,6.2,no,no,no,good,no,no,notckd
397,397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,49.0,6600.0,5.4,no,no,no,good,no,no,notckd
398,398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,51.0,7200.0,5.9,no,no,no,good,no,no,notckd


In [5]:
data.reset_index(drop=True,inplace=True)
data.drop(['id'],axis=1,inplace=True)

In [6]:
data[data.columns[data.dtypes=='object']]

,rbc,pc,pcc,ba,htn,dm,cad,appet,pe,ane,classification
0,normal,abnormal,present,notpresent,yes,no,no,poor,yes,yes,ckd
1,abnormal,abnormal,present,notpresent,yes,yes,no,poor,no,yes,ckd
2,abnormal,abnormal,present,notpresent,yes,yes,no,poor,yes,no,ckd
3,normal,abnormal,present,present,yes,yes,yes,poor,yes,no,ckd
4,abnormal,abnormal,notpresent,notpresent,yes,yes,yes,poor,yes,yes,ckd
...,...,...,...,...,...,...,...,...,...,...,...
153,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd
154,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd
155,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd
156,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd


In [8]:
TrainTest=TrainTestGenerator(1,test_size=0.3,data=data)
for train,test in TrainTest:
    X_train = np.array(data.drop(['classification'],axis=1))[train]
    X_test  = np.array(data.drop(['classification'],axis=1))[test]
    y_train = np.array(data.classification)[train]
    y_test  = np.array(data.classification)[test]

In [9]:
from sklearn.model_selection import KFold,GridSearchCV
kf = KFold(n_splits=4, random_state=None, shuffle=True)

In [11]:
from catboost import CatBoostClassifier
parameters = {'depth': [6,8,10],'learning_rate' : [0.01, 0.05, 0.1],'iterations'    : [30, 50, 100]}

grid_perso = GridSearchHyperParamsCV(model=CatBoostClassifier(), parameters = parameters, cv_splitter = kf, n_jobs=-1 ,verbose=10)

grid = GridSearchCV(estimator=CatBoostClassifier(), param_grid = parameters, cv = kf, n_jobs=-1)

pipe_perso = Pipeline([('cat_trans', CategoricalTransformer(strategy='ordinal_encoding')), ('catboost', grid_perso)])

pipe = Pipeline([('cat_trans', CategoricalTransformer(strategy='ordinal_encoding')), ('catboost', grid)])

pipe_perso.fit(X_train,y_train)

pipe.fit(X_train,y_train)

print(pipe_perso.score(X_test,y_test))
print(pipe_perso['catboost']._best_params)



print(pipe.score(X_test,y_test))
print(pipe['catboost'].best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1378s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  20 out of  27 | elapsed:    2.5s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    3.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1452s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  20 out of  27 | elapsed:    2.0s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    3.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1

0:	learn: 0.6034818	total: 1.64ms	remaining: 47.7ms
1:	learn: 0.5361886	total: 11.1ms	remaining: 156ms
2:	learn: 0.4703068	total: 13.6ms	remaining: 123ms
3:	learn: 0.4038943	total: 15.2ms	remaining: 98.6ms
4:	learn: 0.3622810	total: 23.7ms	remaining: 118ms
5:	learn: 0.3311578	total: 28.1ms	remaining: 112ms
6:	learn: 0.2992620	total: 32.6ms	remaining: 107ms
7:	learn: 0.2753169	total: 40.8ms	remaining: 112ms
8:	learn: 0.2577957	total: 51.8ms	remaining: 121ms
9:	learn: 0.2413906	total: 57.2ms	remaining: 114ms
10:	learn: 0.2127720	total: 58.2ms	remaining: 101ms
11:	learn: 0.1952337	total: 59.7ms	remaining: 89.5ms
12:	learn: 0.1772613	total: 60.9ms	remaining: 79.6ms
13:	learn: 0.1592968	total: 62.5ms	remaining: 71.4ms
14:	learn: 0.1475413	total: 70.8ms	remaining: 70.8ms
15:	learn: 0.1399025	total: 79.9ms	remaining: 69.9ms
16:	learn: 0.1348561	total: 89.4ms	remaining: 68.3ms
17:	learn: 0.1249673	total: 93.7ms	remaining: 62.4ms
18:	learn: 0.1191740	total: 103ms	remaining: 59.7ms
19:	learn: 0.

In [44]:
ct=CategoricalTransformer(strategy='ordinal_encoding')
fs=FeatureSelectorCorr(0.8)

a=ct.transform(X_train)
print(a[0:10])

[[43 80 1.025 0 0 0 0 0 0 81 46 0.6 135 4.9 13.9 48 6900 4.9 0 0 0 0 0 0]
 [57 80 1.025 0 0 0 0 0 0 119 17 1.2 135 4.7 15.4 42 6200 6.2 0 0 0 0 0 0]
 [73 80 1.02 2 0 1 1 0 0 253 142 4.6 138 5.8 10.5 33 7200 4.3 1 1 1 0 0 0]
 [43 60 1.025 0 0 0 0 0 0 108 25 1.0 144 5.0 17.8 43 7200 5.5 0 0 0 0 0 0]
 [29 80 1.02 0 0 0 0 0 0 70 16 0.7 138 3.5 13.7 54 5400 5.8 0 0 0 0 0 0]
 [34 60 1.02 0 0 0 0 0 0 91 49 1.2 135 4.5 13.5 48 8600 4.9 0 0 0 0 0 0]
 [59 70 1.025 0 0 0 0 0 0 130 39 0.7 147 4.7 13.5 46 6700 4.5 0 0 0 0 0 0]
 [61 70 1.025 0 0 0 0 0 0 120 29 0.7 137 3.5 17.4 52 7000 5.3 0 0 0 0 0 0]
 [48 80 1.025 4 0 0 1 0 0 95 163 7.7 136 3.8 9.8 32 6900 3.4 1 0 0 0 0 1]
 [47 80 1.025 0 0 0 0 0 0 124 44 1.0 140 4.9 14.9 41 7000 5.7 0 0 0 0 0 0]]


In [48]:
b = pd.DataFrame(a)
b.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,43,80,1.025,0,0,0,0,0,0,81,...,13.9,48,6900,4.9,0,0,0,0,0,0
1,57,80,1.025,0,0,0,0,0,0,119,...,15.4,42,6200,6.2,0,0,0,0,0,0
2,73,80,1.02,2,0,1,1,0,0,253,...,10.5,33,7200,4.3,1,1,1,0,0,0
3,43,60,1.025,0,0,0,0,0,0,108,...,17.8,43,7200,5.5,0,0,0,0,0,0
4,29,80,1.02,0,0,0,0,0,0,70,...,13.7,54,5400,5.8,0,0,0,0,0,0


In [51]:
b.dtypes

0     object
1     object
2     object
3     object
4     object
5     object
6     object
7     object
8     object
9     object
10    object
11    object
12    object
13    object
14    object
15    object
16    object
17    object
18    object
19    object
20    object
21    object
22    object
23    object
dtype: object

In [49]:
b.corr()

""


In [45]:
def t(X, y=None):
    X = pd.DataFrame(X)
    corrMatrix = X.corr()
    print(corrMatrix)

    corrMatrix = corrMatrix.unstack()
    top_corr = list(corrMatrix[corrMatrix >= 0.1].index)

    to_drop = []
    to_keep = list(X.columns)
    for e in top_corr:
        print(e)
        if (e[0] in to_keep) and (e[1] in to_keep) and (e[0] != e[1]):
            to_drop.append(e[1])
            to_keep.remove(e[1])
    self._dropped_features = to_drop
    X = X.drop(to_drop, axis=1)
    return X.values

In [46]:
a=t(a)

Empty DataFrame
Columns: []
Index: []


IndexError: list index out of range

In [13]:
from sklearn.svm import SVC
parameters =  {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }

grid_perso = GridSearchHyperParamsCV(model=SVC(), parameters = parameters, cv_splitter = kf, n_jobs=-1 ,verbose=10)

grid = GridSearchCV(estimator=SVC(), param_grid = parameters, cv = kf, n_jobs=-1)

pipe_perso = Pipeline([('cat_trans', CategoricalTransformer(strategy='ordinal_encoding')), ('svc', grid_perso)])

pipe = Pipeline([('cat_trans', CategoricalTransformer(strategy='ordinal_encoding')), ('svc', grid)])

pipe_perso.fit(X_train,y_train)

pipe.fit(X_train,y_train)

print(pipe_perso.score(X_test,y_test))
print(pipe_perso['svc']._best_params)



print(pipe.score(X_test,y_test))
print(pipe['svc'].best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0061s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0116s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0042s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0163s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  23 out

0.8085106382978723
{'C': 1000.0, 'gamma': 0.0001}
0.8085106382978723
{'C': 1000.0, 'gamma': 0.0001}
